# step 1  ---import package

In [1]:
import codecs
import time
import re
import pymongo
from pyspark.mllib.feature import HashingTF, IDF
from pymongo import MongoClient 
import findspark
import re
#from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
findspark.init()
import pyspark
import math
import time
from operator import add

#sc = pyspark.SparkContext(appName="myAppName")#myAppName
from __future__ import print_function
from pyspark import SparkContext
#reference  https://www.zybuluo.com/jewes/note/35032

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="myAppName")

# step 2  ---import dictionary

In [2]:
jieba.set_dictionary('/home/cloudera/Desktop/project/dics/dict.txt.big.txt')   #切換至中文繁體字庫
jieba.load_userdict('/home/cloudera/Desktop/project/dics/dict_keyw.txt')       #加入自建詞庫
jieba.load_userdict('/home/cloudera/Desktop/project/dics/dict_keyw_new.txt')

Building prefix dict from /home/cloudera/Desktop/project/dics/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.ubd40686f1bbfd4d132e1423888a99ba6.cache
Loading model cost 0.561 seconds.
Prefix dict has been built succesfully.


# step 3  ---get data from mongoDB

In [25]:
tag1 = ""
#tag2 = ":"
#預設就是自己
client = MongoClient('10.120.28.6',27017)
database = client['Project']
collection =database['news']

#database = client['test_new1']
#collection =database['test1']

#querry_resp = collection.find({"$and":[
#            {"content":{"$regex":tag1}},
           # {"content":{"$regex":tag2}},
#        ]})

querry_resp = collection.find().limit(6000)
print ((querry_resp).count())
#print (type(querry_resp))
#print "查詢結果的第一筆資料標題:",querry_resp[0]["title"]
#print "找到都少筆數:", querry_resp.count()
#print (len(list(querry_resp)))
#for i in querry_resp:
    #print (i['title'])

1680176


In [26]:
write_file_1=time.clock()  

i=0
with codecs.open("/home/cloudera/Desktop/project/practice/project_TFIDF_0520_6000.json",'w',encoding='utf-8') as f:
    for post in querry_resp: 
        i=i+1
        writer=[]
        content=[]
        #清理文章中的符號
        for ele in post['content']:
            fil = re.sub("[\s+\.\!\/_,$%^*「」(+\"\']+|[+——！，。？、~@#￥%……&*（）]+".decode("utf8"), "".decode("utf8"),ele[0])
            if fil =="":
                pass
            else:
                content.append(ele)
                #print (ele)  # show keywords and count
        separator=''
        joined_content=separator.join(content)  
        
        #print (joined_content)
        writer =  (str(post['_id']) + "|" +joined_content)
        #writer = ( "{" + str(post['_id']) + ":" + post['content'] + "}")
        #writer = post['content']
        #print  type(summary)
        #writer_A=(' '.join(jieba.cut(post['content'])))  #把一篇文章切完詞後 再用 "/" 接起來放在一起!    
        #writer_B= str(post['_id'])
        #writer_c= ( "{" + writer_B + ":" + writer_A + "}")
        writer1=writer+'\n'
        f.write(writer1) 
write_file_2=time.clock()  
print  ("共有多少個新聞寫入" , i) 
write_file=write_file_2-write_file_1 
print  ("寫檔案時間" , write_file) 

共有多少個新聞寫入 6000
寫檔案時間 10.46


# step 4  ---TFIDF

## step 4-1 ---- 計算總特徵數

In [3]:
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF_0520_6000.json")
collection_count=documents.count()
print (collection_count)
#每一個元素以flatMap的方式斷詞  ex, (u'\u6843\u5712', 1)
documents_count_feature=documents.map(lambda line: (line.split("|")[1])).flatMap(lambda x: jieba.cut(x)).map(lambda x :(x,1))
documents_count_feature1=documents_count_feature.reduceByKey(lambda x,y : x+y)
count_features=documents_count_feature1.count()
print (count_features)

6000
181391


## step 4-2 ---- 計算TFIDF

In [7]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf().set("spark.driver.maxResultSize", "2g"))
# 加大spark輸出結果用的記憶體大小(不是運算用的)
#sc = pyspark.SparkContext(master="local[*]", conf=conf, appName="kmeans")

time_start_TFIDF= time.clock()

#讀取檔案，以"\n"為一個元素
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF_0520_6000.json")
#print (documents.collect())

#總共元素數量
documents_count=documents.count() 
#print (type(documents_count))

#印出(文章id , keyword)為一個元素的RDD. ex,(u'571787513f3dfa38f08e7009', u'\u7e3d\u7d71')
wordRDD1 = documents.map(lambda line: (line.split("|")[0],line.split("|")[1]))\
.flatMapValues(lambda x: jieba.cut(x))   #.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(add).groupByKey().mapValues(list)  
#print (wordRDD1.collect())

#每一篇文章中的總字詞數
wordRDD1_totalKeywords_Article=wordRDD1.map(lambda x: ((x[0].encode('utf-8')),1)).reduceByKey(add)
#print (wordRDD1_totalKeywords_Article.collect())

#一個關鍵字在此篇文章中出現的總次數
wordRDD1_wordcount_TF = documents.map(lambda line: (line.split("|")[0].encode('utf-8'),line.split("|")[1]))\
.flatMapValues(lambda x: jieba.cut(x)).map(lambda x: ((x[0].encode('utf-8'), x[1]), 1)).reduceByKey(add).groupByKey().mapValues(list)
#print (wordRDD1_wordcount_TF.collect())

#一個關鍵字出現在幾篇文章中    
wordRDD1_wordcount_IDF = documents.map(lambda line: (line.split("|")[0].encode('utf-8'),line.split("|")[1]))\
.flatMapValues(lambda x: jieba.cut(x)).map(lambda x: ((x[0].encode('utf-8'), x[1]), 1)).reduceByKey(add)\
.groupByKey().mapValues(list).map(lambda x:(x[0][1],1)).reduceByKey(add).groupByKey().mapValues(list)    
#print (wordRDD1_wordcount_IDF.collect())

#一個關鍵字出現在幾篇文章中的broadcast前置作業"-------------"
dict_total_word_count=wordRDD1_wordcount_IDF.collect()
dict_total_word_count1=dict(dict_total_word_count)
#print (dict_total_word_count1)

#每一篇文章中的總字詞數broadcast前置作業"-------------"
dict_total_count=wordRDD1_totalKeywords_Article.collect()
dict_total_count1=dict(dict_total_count)
#print (dict_total_count1)    #ok

#broadcast會將此份字典傳到每一個slave中
broadcastVar_1= sc.broadcast(dict_total_count1)
broadcastVar_2= sc.broadcast(dict_total_word_count1)

#s='571787513f3dfa38f08e700c'
#print  (help(broadcastVar))
#print (type(s))
#print (broadcastVar.value[s])   #ok


def my_func_TFIDF(letter):
    article_id=letter[0][0]   #文章的id
    keyword=letter[0][1]       #keyword
    keyword_count_article=(letter[1])[0]   #一個關鍵字在此篇文章中出現的總次數
    words_count_article=int(broadcastVar_1.value[article_id])  #查詢該文章的總字詞數
    keywords_count_article=(broadcastVar_2.value[keyword])[0]  #查詢該關鍵字在多少篇文章中出現
    TF_keyword=(keyword_count_article)/float(words_count_article) #計算詞頻(TF)
    IDF_keyword=(np.log(float(documents_count) / float(keywords_count_article))+ 1.0)  #計算詞頻逆向文件頻率(IDF)
    TFIDF_word=TF_keyword*IDF_keyword     #TF*IDF
    #test=math.log10(float(3) / float(1) )
    #test=np.log(float(3) / float(1))
    #IDF_keyword=np.log(documents_count/)
    return ((article_id,keyword),TFIDF_word)

#print (wordRDD1_wordcount.keys().map(lambda line: (line[0])).collect())    #key
#print (wordRDD1_wordcount_IDF.collect())
#JustDo_TF=wordRDD1_wordcount.keys().map(lambda line: (line[0])).map(my_func)#.values()

#傳入元素樣貌→ (pair keys (文章id , 關鍵字) : value 在此文章中出現的總次數 )
# ex.  (('571787513f3dfa38f08e700c', u'\u9080\u8acb'), [1])


JustDo_TFIDF=wordRDD1_wordcount_TF.map(lambda x: my_func_TFIDF(x))#.values()  
#print (JustDo_TFIDF.collect())             


#TFIDF_list=JustDo_TFIDF.collect()
#print (TFIDF_list)
print ("TFIDF-DIY")
print ("查詢結果資料總筆數:" ,collection_count)
print ("特徵值數量:" , count_features)


TFIDF_list=JustDo_TFIDF.collect()
time_end_TFIDF= time.clock()        
time_for_TFIDF= time_end_TFIDF-time_start_TFIDF
print ("計算tf-idf的時間" , time_for_TFIDF,"seconds")
#print (len(TFIDF_list))


TFIDF-DIY
查詢結果資料總筆數: 6000
特徵值數量: 181391
計算tf-idf的時間 1.89 seconds


# step 5  ---演算結果印出

In [10]:
print ("查詢結果資料總筆數:" ,collection_count)
print ("特徵值數量:" , count_features)
print ("計算tf-idf的時間" , time_for_TFIDF,"seconds")

查詢結果資料總筆數: 32897
特徵值數量: 451056
計算tf-idf的時間 0.03 seconds


In [11]:
print (time_end_TFIDF)
print (time_start_TFIDF)

75.64
75.61


In [ ]:
TFIDF_list=JustDo_TFIDF.collect()
print (len(TFIDF_list))

#TFIDF_list.saveAsTextFile("file:/home/cloudera/Desktop/project/output")

In [15]:
TFIDF_list=JustDo_TFIDF.collect()
for i in TFIDF_list:
    print (i)

# HashTF, IDF

In [94]:
##count features
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF_0514.json")
collection_count=documents.count()

documents_count_feature=documents.map(lambda line: (line.split("|")[1]) ).flatMap(lambda x: jieba.cut(x)).map(lambda x :(x,1))
documents_count_feature1=documents_count_feature.reduceByKey(lambda x,y : x+y)
count_features=documents_count_feature1.count()
print (count_features)

220604


In [8]:
time_start_TFIDF
documents = sc.textFile("file:/home/cloudera/Desktop/project/practice/project_TFIDF_0520_6000.json")
collection_count=documents.count()
#print (collection_count)

print  ("查詢結果資料總筆數:" ,collection_count)


time_start_jieba = time.clock()
documents1=documents.map(lambda line: (line.split("|")[0],line.split("|")[1])).flatMapValues(lambda x: jieba.cut(x)).groupByKey().mapValues(list).map(lambda x: ( x[1]))
#print (documents1.collect())
time_end_jieba = time.clock()

#print (documents1.collect())

time_for_jieba= time_end_jieba-time_start_jieba

#print ("I/O time + jieba的時間:" ,time_for_jieba,"seconds")
print ("hashTF,IDF")
print ("特徵值數量:" , count_features)

#print (documents.collect())
#wordRDD1 = documents.flatMapValues(lambda x: jieba.cut(x))#.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(add)#.groupByKey().mapValues(list)     #append('/'.join(jieba.cut(summary)))                                                                                                                                                                                                                                                                           
#wordRDD2 =wordRDD1.groupByKey().mapValues(list)
#wordRDD3=wordRDD2.map(lambda x: ( x[1]))
#print (wordRDD2.collect())

#time_start_TFIDF = time.clock()
hashingTF = HashingTF()
tf = hashingTF.transform(documents1)
tf.cache()
#print ("--------------------------")
#print ("tf:")
#print (tf.collect()) 
idf = IDF().fit(tf)
tfidf = idf.transform(tf)   #<class 'pyspark.mllib.feature.IDFModel'>
time_end_TFIDF= time.clock()

time_for_TFIDF= time_end_TFIDF-time_start_TFIDF
print ("計算tf-idf的時間" , time_for_TFIDF,"seconds")

#print ("--------------------------")
#print ("tfidf:")
#print (tfidf.collect()) 

#combined = documents.zip(tfidf)
#print ("--------------------------")
#print (combined.collect()) 

查詢結果資料總筆數: 6000
hashTF,IDF
特徵值數量: 181391
計算tf-idf的時間 1.95 seconds
